In [0]:
!pip install torch torchvision

In [0]:
import torch, torchvision
from torch import nn,optim
from torch.autograd import Variable as var 


In [0]:
n_batch = 64
learning_rate = 0.01
n_epoch = 3
n_print = 10

In [4]:
#### Part I : Loading Your Data 

T = torchvision.transforms.ToTensor()
train_data = torchvision.datasets.MNIST('mnist_data',train=True,download=True,transform=T)
val_data = torchvision.datasets.MNIST('mnist_data',train=False,download=True,transform=T)

train_dl = torch.utils.data.DataLoader(train_data,batch_size = n_batch)
val_dl = torch.utils.data.DataLoader(val_data,batch_size = n_batch)

  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 27676732.96it/s]                            


Extracting mnist_data/MNIST/raw/train-images-idx3-ubyte.gz


32768it [00:00, 452748.18it/s]
  1%|          | 16384/1648877 [00:00<00:11, 146881.37it/s]

Extracting mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 7680439.39it/s]                            
8192it [00:00, 186483.32it/s]


Extracting mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [0]:
#### Part II : Writing the Network
class myCNN(nn.Module):
  def __init__(self):
    super(myCNN,self).__init__()
    self.cnn1 = nn.Conv2d(in_channels = 1, out_channels = 3, kernel_size = 3)
    self.cnn2 = nn.Conv2d(in_channels = 3, out_channels = 2, kernel_size = 5)
    self.linear = nn.Linear(968,10)
    self.relu = nn.ReLU()
  
  def forward(self,x):
    n = x.size(0)
    x = self.relu(self.cnn1(x))
    x = self.relu(self.cnn2(x))
    x = x.view(n,-1)
    x = self.linear(x)
    return x

In [11]:
#### Part III : Writing the main Training loop

mycnn = myCNN().cuda()
cec = nn.CrossEntropyLoss()
optimizer = optim.Adam(mycnn.parameters(),lr = learning_rate)

def validate(model,data):
  # To get validation accuracy = (correct/total)*100.
  total = 0
  correct = 0
  for i,(images,labels) in enumerate(data):
    images = var(images.cuda())
    x = model(images)
    value,pred = torch.max(x,1)
    pred = pred.data.cpu()
    total += x.size(0)
    correct += torch.sum(pred == labels)
  return correct*100./total

for e in range(n_epoch):
  for i,(images,labels) in enumerate(train_dl):
    images = var(images.cuda())
    labels = var(labels.cuda())
    optimizer.zero_grad()
    pred = mycnn(images)
    loss = cec(pred,labels)
    loss.backward()
    optimizer.step()
    if (i+1) % n_print == 0:
      accuracy = float(validate(mycnn,val_dl))
      print('Epoch :',e+1,'Batch :',i+1,'Loss :',float(loss.data),'Accuracy :',accuracy,'%')

Epoch : 1 Batch : 10 Loss : 2.303971290588379 Accuracy : 21.0 %
Epoch : 1 Batch : 20 Loss : 2.243605613708496 Accuracy : 29.0 %
Epoch : 1 Batch : 30 Loss : 2.206059217453003 Accuracy : 39.0 %
Epoch : 1 Batch : 40 Loss : 2.183595657348633 Accuracy : 48.0 %
Epoch : 1 Batch : 50 Loss : 2.1403331756591797 Accuracy : 52.0 %
Epoch : 1 Batch : 60 Loss : 2.074362277984619 Accuracy : 56.0 %
Epoch : 1 Batch : 70 Loss : 2.052757978439331 Accuracy : 59.0 %
Epoch : 1 Batch : 80 Loss : 1.9684913158416748 Accuracy : 62.0 %
Epoch : 1 Batch : 90 Loss : 1.9605399370193481 Accuracy : 65.0 %
Epoch : 1 Batch : 100 Loss : 1.8538439273834229 Accuracy : 67.0 %
Epoch : 1 Batch : 110 Loss : 1.804017186164856 Accuracy : 68.0 %
Epoch : 1 Batch : 120 Loss : 1.7926627397537231 Accuracy : 69.0 %
Epoch : 1 Batch : 130 Loss : 1.677207350730896 Accuracy : 70.0 %
Epoch : 1 Batch : 140 Loss : 1.5213065147399902 Accuracy : 71.0 %
Epoch : 1 Batch : 150 Loss : 1.614527702331543 Accuracy : 72.0 %
Epoch : 1 Batch : 160 Loss :

In [7]:
!nvidia-smi

Mon May 27 05:51:55 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    29W /  70W |    801MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  